In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

In [3]:
# Load Data
train = pd.read_csv('/kaggle/input/fake-news/train.csv').fillna('')
test = pd.read_csv('/kaggle/input/fake-news/test.csv').fillna('')

In [4]:
print(train.shape)
print(test.shape)

(20800, 5)
(5200, 4)


In [5]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [7]:
train.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [8]:
test.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

In [9]:
# Merge 'author' and 'title'
X_train = (train['author'] + ' ' + train['title']).apply(lambda x: re.sub('[^a-zA-Z]', ' ', x.lower()).split())
X_test = (test['author'] + ' ' + test['title']).apply(lambda x: re.sub('[^a-zA-Z]', ' ', x.lower()).split())

In [10]:
# Separate labels
Y_train = train['label'].values

In [11]:
# Stemming
porter_stemmer = PorterStemmer()
X_train = X_train.apply(lambda x: ' '.join([porter_stemmer.stem(word) for word in x if word not in stopwords.words('english')]))
X_test = X_test.apply(lambda x: ' '.join([porter_stemmer.stem(word) for word in x if word not in stopwords.words('english')]))

In [12]:
# Vectorizing
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [13]:
# SVM Model
classifier = SVC(kernel="linear")
classifier.fit(X_train, Y_train)

SVC(kernel='linear')

In [14]:
# Prediction
predictions = classifier.predict(X_test)

In [15]:
# Save to CSV
output = pd.DataFrame({'id': test.id, 'label': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
